#### Neural Networks
使用torch.nn包来构建神经网络
上一讲已经讲过了autograd，nn包依赖autograd包来定义模型并求导。
一个nn.Module包括各个层和一个forward(input)方法，该方法返回output
例如：
![](https://raw.githubusercontent.com/CallMeNumber1/FigureBed/windows/img/20190917211507.png)
是一个简单的前馈神经网络，它接收一个输入，然后一层接着一层传递，最后输出计算的结果。
神经网络的典型训练过程如下：
1. 定义一些可学习的参数（权重）神经网络模型；
2. 在数据集上迭代
3. 通过神经网络处理输入
4. 计算损失（输出结果和正确值的差值大小）
5. 将梯度反向传播回网络的参数
6. 更新网络的参数，主要使用如下简单的更新规则：$weight = weight - learning rate * gradient$
> 事实上，上面即为LeNet-5（CNN的开山之作），麻雀虽小，五脏俱全
>- 用卷积层提取空间特征
>- 由空间平均得到子样本
>- 由tanh或sigmoid得到非线性
>- 用multi-layer nerual network（MLP）作为最终分类器
>- 层层之间用稀疏的连接矩阵，以避免大的计算成本

> 个人理解：即卷积层提取特征，最后再用多层神经网络进行分类

####  定义网络

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    # 定义Net的初始化函数，这个函数定义了该神经网络的基本结构
    def __init__(self):
        # 复制并使用Net的父类的初始化方法，即运行nn.Module的初始化函数
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5 * 5 squre convolution kernel
        # 定义conv1函数的是图像卷积函数：输入为图像（1个频道，即灰度图）,输出为 6张特征图, 卷积核为5x5正方形
        self.conv1 = nn.Conv2d(1, 6, 5)
        # 定义conv2函数的是图像卷积函数：输入为6张特征图,输出为16张特征图, 卷积核为5x5正方形
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        # 定义fc1（fullconnect）全连接函数1为线性函数：y = Wx + b，并将16*5*5个节点连接到120个节点上
        self.fc1 = nn.Linear(16 * 5 *5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    # 定义该神经网络的前向传播函数，该函数必须定义，一旦定义成功，反向传播函数也会自动生成（autograd）
    def forward(self, x):
        # Max pooling over a (2, 2) window
        # 输入x经过卷积conv1之后，经过激活函数ReLU，使用2x2的窗口进行最大池化Max pooling，然后更新到x。
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        # view函数将张量x变形成一维的向量形式，总特征数并不改变，为接下来的全连接作准备。
        # 使用num_flat_features函数计算张量x的总特征量（把每个数字都看出是一个特征，即特征总量），比如x是4*2*2的张量，那么它的特征总量就是16
        x = x.view(-1, self.num_flat_features(x))
        # 输入x经过全连接1，再经过ReLU激活函数，然后更新x
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        # 输入x经过全连接3，然后更新x
        x = self.fc3(x)
        return x
    def num_flat_features(self, x):
        # all dimensions except the batch dimension
        size = x.size()[1:] 
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


net.parameters()返回可被学习的参数（权重）列表和值

In [2]:
params = list(net.parameters())
print(len(params))
# conv1's weight
print(params[0].size())

10
torch.Size([6, 1, 5, 5])


测试随机输入32 * 32。注：这个网络（LeNet）期望的输入大小是32 * 32，如果使用MINIST数据集来训练这个网络，请把图片大小重新调整到32 * 32

In [3]:
# 为何为1 * 1 * 32 * 32 而非直接32 * 32 呢？
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[-0.0529,  0.0584,  0.0137,  0.0311,  0.0697, -0.1507, -0.0106,  0.0840,
          0.0604,  0.0690]], grad_fn=<AddmmBackward>)


将所有参数的梯度缓存清零，然后进行随机梯度的反向传播

In [4]:
net.zero_grad()
out.backward(torch.randn(1, 10))

>``torch.nn`` 只支持小批量输入。整个 ``torch.nn`` 包都只支持小批量样本，而不支持单个样本。 例如，``nn.Conv2d`` 接受一个4维的张量， ``每一维分别是sSamples * nChannels * Height * Width（样本数*通道数*高*宽）``。 如果你有单个样本，只需使用 ``input.unsqueeze(0)`` 来添加其它的维数

在继续之前，我们回顾一下到目前为止用到的类
- torch.Tensor：一个用自动调用 backward()实现支持自动梯度计算的 多维数组 ， 并且保存关于这个向量的梯度
- nn.Module：神经网络模块。封装参数、移动到GPU上运行、导出、加载等
- nn.Parameter：一种变量，当把它赋值给一个Module时，被自动注册为一个参数
- autograd.Function：实现一个自动求导操作的前向和反向定义...

重点如下：
- 定义一个网络
- 处理输入，调用backward
还剩下：
- 计算损失
- 更新网络权重
### 损失函数
一个损失函数接收一对(output, target)作为输入，计算一个值来估计网络的输出和目标值差多少
nn包中有很多不同的损失函数。 nn.MSELoss是一个比较简单的损失函数，它计算输出和目标间的均方误差， 例如：

In [6]:
output = net(input)
# 随机值作为样例
target = torch.randn(10)
# 使target和output的shape相同
target = target.view(1, -1)
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(1.3177, grad_fn=<MseLossBackward>)


现在，如果在反向过程中跟随loss，使用它的.grad_fn属性，将会看到如下所示的计算图：
input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
      -> view -> linear -> relu -> linear -> relu -> linear
      -> MSELoss
      -> loss
所以，当我们调用 loss.backward()时,整张计算图都会 根据loss进行微分，而且图中所有设置为requires_grad=True的张量 将会拥有一个随着梯度累积的.grad 张量。

### 反向传播
调用loss.backward()获得反向传播的误差

但是在调用前需要清除已存在的梯度，否则梯度将被累加到已存在的梯度。

现在，我们将调用loss.backward()，并查看conv1层的偏差（bias）项在反向传播前后的梯度。

In [7]:
# 清除梯度
net.zero_grad()
print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)
loss.backward()
print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0012, -0.0012, -0.0160,  0.0099,  0.0057,  0.0078])


如何使用损失函数

**稍后阅读**：nn包，包含了各种用来构成深度神经网络构建块的模块和损失函数
### 更新权重
在实践中最简单的权重更新规则是随机梯度下降（SGD）：
`weight = weight - learning_rate * gradient`
我们可以使用简单的Python代码来实现这个规则：：
```python
learning_rate = 0.01
    for f in net.parameters():
        f.data.sub_(f.grad.data * learning_rate)
```
但是当使用神经网络是想要使用各种不同的更新规则时，比如SGD、Nesterov-SGD、Adam、RMSPROP等，PyTorch中构建了一个包torch.optim实现了所有的这些规则。 使用它们非常简单：

In [11]:
import torch.optim as optim
# create your optimizer
optimizer = optim.SGD(net.parameters(), lr = 0.01)

# in your training loop
optimizer.zero_grad()  # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()      # Does the update

>注意： 观察如何使用``optimizer.zero_grad()``手动将梯度缓冲区设置为零。
  这是因为梯度是按Backprop部分中的说明累积的。

### 补充：本节涉及到的Python知识
**类的定义**
Python是一门动态语言，因此它的类所包含的类变量/实例变量可以动态增加或删除
在类中定义的方法默认是实例方法，定义实例方法的方法与定义函数的方法基本相同，只是实例方法的第一个参数会被绑定到方法的调用者（该类的实例），因此实例方法至少应该定义一个参数，该参数通常会被命名为 self。
>注意：实例方法的第一个参数并不一定要叫 self，其实完全可以叫任意参数名，只是约定俗成地把该参数命名为 self，这样具有最好的可读性。

在实例方法中有一个特别的方法：`__init__`，这个方法被称为构造方法。构造方法用于构造该类的对象，Python 通过调用构造方法返回该类的对象（无须使用 new）。|
>Python 中很多这种以双下划线开头、双下划线结尾的方法，都具有特殊的意义，

构造方法是一个类创建对象的根本途径，因此 Python 还提供了一个功能：如果开发者没有为该类定义任何构造方法，那么 Python 会自动为该类定义一个只包含一个 self 参数的默认的构造方法
**类的使用**
定义变量；创建对象；派生子类
### 其他
1.nn.Conv2d(self, in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True))\n
- in_channel:　输入数据的通道数，例RGB图片通道数为3；
- out_channel: 输出数据的通道数，这个根据模型调整；
- kennel_size: 卷积核大小，可以是int，或tuple；kennel_size=2,意味着卷积大小(2,2)， kennel_size=（2,3），意味着卷积大小（2，3）即非正方形卷积
- stride：步长，默认为1，与kennel_size类似，stride=2,意味着步长上下左右扫描皆为2， stride=（2,3），左右扫描步长为2，上下为3；
- padding：　零填充

2.nn.Linear(in_features, out_feartures, bias = True)
- in_features：输入样本的大小
- out_features:输出样本的大小
- bias: 如果设置为False，则图层不会学习附加偏差。默认值：True

对传入数据应用线性变换：y = Ax + b

weight = Parameter(torch.Tensor(out_features, in_features))

bias = Parameter(torch.Tensor(out_features))

3.x.view()
把原来tensor中的数据按照行优先的顺序排成一个一维的数据，然后按照参数组合成其他维度的tensor

>另外，参数不可空。参数中的-1就代表这个位置由其他位置的数字来推断，只要在不歧义的情况下就可以推断出来

4.self.num_flat_features(x)
使用num_flat_features函数计算张量x的总特征量（把每个数字都看出是一个特征，即特征总量），比如x是4 * 2 * 2的张量，那么它的特征总量就是16

In [ ]:
m = nn.Linear(20, 30)
# weight: [30, 20] bias: 30
input = torch.randn(128, 20)
output = m(input)
print(output.size())

In [ ]:
a = torch.Tensor([[1, 2, 3], [4, 5, 6]])
b = torch.Tensor([1, 2, 3, 4, 5, 6])
print(a.view(1, 6))
print(b.view(1, 6))